In [1]:
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from collections import deque

# Grid setup
ROWS, COLS = 6, 6
start = (0, 0)
goal = (5, 5)
obstacles = {(1, 1), (2, 2), (3, 3), (3, 2), (2, 3)}

def draw_grid(path=set(), frontier1=set(), frontier2=set(), visited1=set(), visited2=set(), meeting=None):
    fig, ax = plt.subplots()
    ax.set_xticks(range(COLS+1))
    ax.set_yticks(range(ROWS+1))
    ax.grid(True)
    ax.set_xlim(-0.5, COLS - 0.5)
    ax.set_ylim(-0.5, ROWS - 0.5)
    ax.invert_yaxis()

    for r in range(ROWS):
        for c in range(COLS):
            node = (r, c)
            color = 'white'
            if node in obstacles:
                color = 'black'
            elif node == start:
                color = 'green'
            elif node == goal:
                color = 'gold'
            elif node == meeting:
                color = 'purple'
            elif node in path:
                color = 'blue'
            elif node in frontier1:
                color = 'orange'
            elif node in frontier2:
                color = 'red'
            elif node in visited1:
                color = 'lightgray'
            elif node in visited2:
                color = 'pink'
            ax.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1, color=color))

    plt.title("Bidirectional Search")
    plt.show()
    time.sleep(0.5)
    clear_output(wait=True)

def get_neighbors(node):
    r, c = node
    for dr, dc in [(0,1),(1,0),(-1,0),(0,-1)]:
        nr, nc = r+dr, c+dc
        next_node = (nr, nc)
        if (0 <= nr < ROWS and 0 <= nc < COLS and next_node not in obstacles):
            yield next_node

def bidirectional_search(start, goal):
    queue_start = deque([(start, [start])])
    queue_goal = deque([(goal, [goal])])
    visited_start = {start: [start]}
    visited_goal = {goal: [goal]}

    while queue_start and queue_goal:
        # Expand forward
        current_start, path_start = queue_start.popleft()
        for neighbor in get_neighbors(current_start):
            if neighbor not in visited_start:
                visited_start[neighbor] = path_start + [neighbor]
                queue_start.append((neighbor, path_start + [neighbor]))
                if neighbor in visited_goal:
                    path = visited_start[neighbor] + visited_goal[neighbor][::-1][1:]
                    draw_grid(path=set(path), meeting=neighbor)
                    return path

        # Expand backward
        current_goal, path_goal = queue_goal.popleft()
        for neighbor in get_neighbors(current_goal):
            if neighbor not in visited_goal:
                visited_goal[neighbor] = path_goal + [neighbor]
                queue_goal.append((neighbor, path_goal + [neighbor]))
                if neighbor in visited_start:
                    path = visited_start[neighbor] + visited_goal[neighbor][::-1][1:]
                    draw_grid(path=set(path), meeting=neighbor)
                    return path

        # Visualization step
        draw_grid(
            frontier1={n for n, _ in queue_start},
            frontier2={n for n, _ in queue_goal},
            visited1=set(visited_start.keys()),
            visited2=set(visited_goal.keys())
        )

    return None

# Run it
bidirectional_search(start, goal)


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 5),
 (2, 5),
 (3, 5),
 (4, 5),
 (5, 5)]